Database 1: Samples
===================

After fitting a large suite of strong lens data, we can use the aggregator to load the results and manipulate,
interpret and visualize them using a Python script or Jupyter notebook.

This script uses the results generated by the script `/autolens_workspace/aggregator/phase_runner.py`, which fitted 3
simulated strong lenses with:

 - An `EllipticalIsothermal` `MassProfile`.for the lens galaxy's mass.
 - An `EllipticalSersic` `LightProfile`.for the source galaxy's light.

This fit was performed using one `PhaseImaging` object, and the first four tutorials (a1-a4) cover how to use the
aggregator on the results of `Phase`'s (as opposed to `Pipeline`'s). However, the aggregator API is extremely similar
across both and learning to use the aggregator with phases can be easily applied to the results of pipelines.

__Samples__

If you are familiar with the `Samples` object returned from a *PyAutoLens* model-fit (e.g. via a `Phase` or `Pipeline`)
You will be familiar with most of the content in this script. Nevertheless, the script also describes how to use
the `Aggregator`, so will be useful for you too!

__File Output__

The results of this fit are in the `autolens_workspace/output/aggregator` folder. First, take a look in this folder.
Provided you haven't rerun the runner, you`ll notice that all the results (e.g. samples, samples_backup,
model.results, images, etc.) are in .zip files as opposed to folders that can be instantly accessed.

This is because when the pipeline was run, the `remove_files` option in the `config/general.ini` was set to True.
This means all results (other than the .zip file) were removed. This feature is implemented because super-computers
often have a limit on the number of files allowed per user.

Bare in mind the fact that all results are in .zip files, we'll come back to this point in a second.

In [1]:
%matplotlib inline
from pyprojroot import here
workspace_path = str(here())
%cd $workspace_path
print(f"Working Directory has been set to `{workspace_path}`")

from os import path
import autofit as af

/mnt/c/Users/Jammy/Code/PyAuto/autolens_workspace
Working Directory has been set to `/mnt/c/Users/Jammy/Code/PyAuto/autolens_workspace`


To set up the aggregator we simply pass it the folder of the results we want to load.

In [2]:
agg = af.Aggregator(directory=path.join("output", "database", "phase_runner"))

Aggregator loading phases... could take some time.

 A total of 3 phases and results were found.


Before we continue, take another look at the output folder. The .zip files containing results have now all been 
unzipped, such that the results are accessible on your laptop for navigation. This means you can run fits to many 
lenses on a super computer and easily unzip all the results on your computer afterwards via the aggregator.

To begin, let me quickly explain what a generator is in Python, for those unaware. A generator is an object that 
iterates over a function when it is called. The aggregator creates all objects as generators, rather than lists, or 
dictionaries, or whatever.

Why? Because lists store every entry in memory simultaneously. If you fit many lenses, you`ll have lots of results and 
therefore use a lot of memory. This will crash your laptop! On the other hand, a generator only stores the object in 
memory when it runs the function; it is free to overwrite it afterwards. Thus, your laptop won't crash!

There are two things to bare in mind with generators:

    1) A generator has no length, thus to determine how many entries of data it corresponds to you first must convert 
       it to a list.
    
    2) Once we use a generator, we cannot use it again and we'll need to remake it.

We can now create a `samples` generator of every fit, which creates `Sample`'s objects of our results. This object 
contains information on the result of the non-linear search.

In [3]:
samples_gen = agg.values("samples")

When we print this the length of this generator converted to a list of outputs we see 3 different NestSamples 
instances. These correspond to each fit of each phase to each of our 3 images.

In [4]:
print("NestedSampler Samples: \n")
print(samples_gen)
print()
print("Total Samples Objects = ", len(list(samples_gen)), "\n")

NestedSampler Samples: 


Total Samples Objects =  3 



The `Samples` class contains all the parameter samples, which is a list of lists where:
 
 - The outer list is the size of the total number of samples.
 - The inner list is the size of the number of free parameters in the fit.

In [5]:
for samples in agg.values("samples"):

    print("All parameters of the very first sample")
    print(samples.parameters[0])
    print("The third parameter of the tenth sample")
    print(samples.parameters[9][2])

print("Samples: \n")
print(agg.values("samples"))
print()
print("Total Samples Objects = ", len(list(agg.values("samples"))), "\n")

All parameters of the very first sample
[-0.12567441545243616, 0.12493260013763148, 0.008102700453438324, -0.040471094716060876, 2.6380691939403276, -0.09843466041102081, -0.5411398840588635, 0.22710284450182738, 0.08348303748648811, 272831.4641989508, 26.908605988429557, 7.392337722486532]
The third parameter of the tenth sample
-0.008206109769787688
All parameters of the very first sample
[-0.021040921224626707, 0.15882234778787746, 0.1739863500448214, 0.07551931422328496, 2.647195846745857, -0.015129979461058839, -0.5131706711877874, 0.3893728369053689, 0.4839335576125199, 9426.44985720727, 1.6513532972443314, 6.6983538520222305]
The third parameter of the tenth sample
0.16856347167832308
All parameters of the very first sample
[0.0863515241118672, -0.10428037525961464, -0.007311344533065417, 0.10599021545468672, 2.1184526829163035, 0.27382557460326284, 0.23039997843879537, 0.2050260226464596, -0.23713039567073446, 237333.96229025297, 13.450669780468019, 6.002644986595653]
The third

The `Samples` class contains the log likelihood, log prior, log posterior and weights of every sample, where:

   - The log likelihood is the value evaluated from the likelihood function (e.g. -0.5 * chi_squared + the noise 
     normalization).
    
   - The log prior encodes information on how the priors on the parameters maps the log likelihood value to the log
     posterior value.
      
   - The log posterior is log_likelihood + log_prior.
    
   - The weight gives information on how samples should be combined to estimate the posterior. The weight values 
     depend on the sampler used, for example for MCMC they will all be 1`s.

In [6]:
for samples in agg.values("samples"):
    print("log(likelihood), log(prior), log(posterior) and weight of the tenth sample.")
    print(samples.log_likelihoods[9])
    print(samples.log_priors[9])
    print(samples.log_posteriors[9])
    print(samples.weights[9])

log(likelihood), log(prior), log(posterior) and weight of the tenth sample.
-28004911054775.438
0.9955386646367158
-28004911054774.44
0.0
log(likelihood), log(prior), log(posterior) and weight of the tenth sample.
-93595169443.18277
5.868120375088089
-93595169437.31465
0.0
log(likelihood), log(prior), log(posterior) and weight of the tenth sample.
-6468454536.643861
112.95622092362972
-6468454423.68764
0.0


We can use the outputs to create a list of the maximum log likelihood model of each fit to our three images.

In [7]:
ml_vector = [samps.max_log_likelihood_vector for samps in agg.values("samples")]

print("Max Log Likelihood Model Parameter Lists: \n")
print(ml_vector, "\n\n")

Max Log Likelihood Model Parameter Lists: 

[[0.0005011469424134799, 2.8646028103426006e-05, 0.00033860811559423477, 0.2506258477604136, 0.7995298932765209, 0.1022398094208931, 0.10135167292546636, 0.004174580694160241, 0.24886928393106947, 0.30923220146742353, 0.975337964017732, 1.9838481081252999], [-0.003311352584049788, -0.0013423716208089103, 0.25241976518446474, 0.0014678422656577365, 1.0000944138098908, 0.1994679818483178, 0.201320285261928, -0.0015574458677028063, 0.1511913200163771, 0.3092721782170849, 1.4684245360387678, 2.481949030958151], [0.0007702213115812011, 8.653838377550441e-05, 0.24938436966856445, 3.965353633037923e-05, 1.2001577147256408, 0.30064097037361154, 0.3007341839796714, 0.0012095561222437087, 0.22298649966358602, 0.3004406929450053, 1.988160530607494, 3.005162837220106]] 




This provides us with lists of all model parameters. However, this isn't that much use, which values correspond to 
which parameters?

The list of parameter names are available as a property of the `Model` included with the `Samples`, as are labels 
which can be used for labeling figures.

In [8]:
for samples in agg.values("samples"):
    model = samples.model
    print(model)
    print(model.parameter_names)
    print(model.parameter_labels)

Galaxy (centre_0, GaussianPrior, mean = 0.0, sigma = 0.1), (centre_1, GaussianPrior, mean = 0.0, sigma = 0.1), (elliptical_comps_0, GaussianPrior, mean = 0.0, sigma = 0.3), (elliptical_comps_1, GaussianPrior, mean = 0.0, sigma = 0.3), (einstein_radius, UniformPrior, lower_limit = 0.0, upper_limit = 3.0), Galaxy (centre_0, GaussianPrior, mean = 0.0, sigma = 0.3), (centre_1, GaussianPrior, mean = 0.0, sigma = 0.3), (elliptical_comps_0, GaussianPrior, mean = 0.0, sigma = 0.3), (elliptical_comps_1, GaussianPrior, mean = 0.0, sigma = 0.3), (intensity, LogUniformPrior, lower_limit = 1e-06, upper_limit = 1000000.0), (effective_radius, LogUniformPrior, lower_limit = 0.0001, upper_limit = 30.0), (sersic_index, UniformPrior, lower_limit = 0.5, upper_limit = 8.0), None, None
['centre_0', 'centre_1', 'elliptical_comps_0', 'elliptical_comps_1', 'einstein_radius', 'centre_0', 'centre_1', 'elliptical_comps_0', 'elliptical_comps_1', 'intensity', 'effective_radius', 'sersic_index']
['y', 'x', '\\xi', '

These lists will be used later for visualization, how it is often more useful to create the model instance of every fit.

In [9]:
ml_instances = [samps.max_log_likelihood_instance for samps in agg.values("samples")]
print("Maximum Log Likelihood Model Instances: \n")
print(ml_instances, "\n")

Maximum Log Likelihood Model Instances: 

[<autofit.mapper.model.ModelInstance object at 0x7f91cf7b7130>, <autofit.mapper.model.ModelInstance object at 0x7f91e038f220>, <autofit.mapper.model.ModelInstance object at 0x7f91f13a79a0>] 



A model instance contains all the model components of our fit, most importantly the list of galaxies we specified in 
the pipeline.

In [10]:
print(ml_instances[0].galaxies)
print(ml_instances[1].galaxies)
print(ml_instances[2].galaxies)

These galaxies will be named according to the phase (in this case, `lens` and `source`).

In [11]:
print(ml_instances[0].galaxies.lens)
print()
print(ml_instances[1].galaxies.source)

Redshift: 0.5
Mass Profiles:
EllipticalIsothermal
centre: (0.0005011469424134799, 2.8646028103426006e-05)
elliptical_comps: (0.00033860811559423477, 0.2506258477604136)
axis_ratio: 0.5991990232599627
phi: 0.03870471521885026
einstein_radius: 0.7995298932765209
slope: 2.0
core_radius: 0.0
id: 27
_assertions: []
cls: <class 'autogalaxy.profiles.mass_profiles.total_mass_profiles.EllipticalIsothermal'>

Redshift: 1.0
Light Profiles:
EllipticalSersic
centre: (0.1994679818483178, 0.201320285261928)
elliptical_comps: (-0.0015574458677028063, 0.1511913200163771)
axis_ratio: 0.737318575341367
phi: -0.2950960374929498
intensity: 0.3092721782170849
effective_radius: 1.4684245360387678
sersic_index: 2.481949030958151
id: 94688
_assertions: []
cls: <class 'autogalaxy.profiles.light_profiles.EllipticalSersic'>


Their `LightProfile`'s and `MassProfile`'s are also named according to the phase.

In [12]:
print(ml_instances[0].galaxies.lens.mass)
print(ml_instances[1].galaxies.source.bulge)

EllipticalIsothermal
centre: (0.0005011469424134799, 2.8646028103426006e-05)
elliptical_comps: (0.00033860811559423477, 0.2506258477604136)
axis_ratio: 0.5991990232599627
phi: 0.03870471521885026
einstein_radius: 0.7995298932765209
slope: 2.0
core_radius: 0.0
id: 27
_assertions: []
cls: <class 'autogalaxy.profiles.mass_profiles.total_mass_profiles.EllipticalIsothermal'>
EllipticalSersic
centre: (0.1994679818483178, 0.201320285261928)
elliptical_comps: (-0.0015574458677028063, 0.1511913200163771)
axis_ratio: 0.737318575341367
phi: -0.2950960374929498
intensity: 0.3092721782170849
effective_radius: 1.4684245360387678
sersic_index: 2.481949030958151
id: 94688
_assertions: []
cls: <class 'autogalaxy.profiles.light_profiles.EllipticalSersic'>


We can also access the `median pdf` model, which is the model computed by marginalizing over the samples of every 
parameter in 1D and taking the median of this PDF.

In [13]:
mp_vector = [samps.median_pdf_vector for samps in agg.values("samples")]
mp_instances = [samps.median_pdf_instance for samps in agg.values("samples")]

print("Median PDF Model Parameter Lists: \n")
print(mp_vector, "\n")
print("Most probable Model Instances: \n")
print(mp_instances, "\n")
print(mp_instances[0].galaxies.lens.mass)
print()

Median PDF Model Parameter Lists: 

[[0.0005588773579545048, -0.0004882044531115121, -0.0007754575930409335, 0.2507150993491136, 0.7993976072252871, 0.10212209256699706, 0.1015998170187702, 0.00322667355556284, 0.24865285752790817, 0.3093424427162158, 0.9747078996419276, 1.9839073589244203], [-0.002846927099670824, -0.0009363241594987072, 0.25198136111417135, 0.0007810536033499618, 0.999752977831672, 0.19962902681734987, 0.2013993963317366, -0.002093425724986091, 0.15173018653580883, 0.3072897875038004, 1.4747573799646325, 2.486648503350614], [0.00030944323593424795, -0.0010304387177771642, 0.24853562981090788, -0.0007304992421331382, 1.2002887966780076, 0.30026630045762404, 0.2999558015317526, 0.00033287607013590307, 0.22281025730962672, 0.2996799999818305, 1.99116403204327, 3.010509571914943]] 

Most probable Model Instances: 

[<autofit.mapper.model.ModelInstance object at 0x7f91cf7cb940>, <autofit.mapper.model.ModelInstance object at 0x7f92128befa0>, <autofit.mapper.model.ModelInst

We can compute the model parameters at a given sigma value (e.g. at 3.0 sigma limits).

These parameter values do not account for covariance between the model. For example if two parameters are degenerate 
this will find their values from the degeneracy in the `same direction` (e.g. both will be positive). we'll cover
how to handle covariance in a later tutorial.

Here, I use "uv3" to signify this is an upper value at 3 sigma confidence,, and "lv3" for the lower value.

In [14]:
uv3_vectors = [
    samps.vector_at_upper_sigma(sigma=3.0) for samps in agg.values("samples")
]

uv3_instances = [
    samps.instance_at_upper_sigma(sigma=3.0) for samps in agg.values("samples")
]

lv3_vectors = [
    samps.vector_at_lower_sigma(sigma=3.0) for samps in agg.values("samples")
]

lv3_instances = [
    samps.instance_at_lower_sigma(sigma=3.0) for samps in agg.values("samples")
]

print("Errors Lists: \n")
print(uv3_vectors, "\n")
print(lv3_vectors, "\n")
print("Errors Instances: \n")
print(uv3_instances, "\n")
print(lv3_instances, "\n")

Errors Lists: 

[[0.002974565506564607, 0.0026587231024849806, 0.0033971392546413566, 0.25685662524225, 0.8013505922401404, 0.1040456270407741, 0.10420624996766509, 0.008057726926042457, 0.25331753606556184, 0.32154761120219794, 0.9943484393839246, 2.0284506174034775], [-0.0003648718422998931, 0.0013158697158632666, 0.25413744684366796, 0.0031257699317657904, 1.001476531784457, 0.2009979011103656, 0.20336429666965636, 0.002484754463056009, 0.1554599969382773, 0.31752353424248847, 1.4981345882506634, 2.508949953942122], [0.002417804162511781, 0.001795403528961431, 0.2504294928909334, 0.0017291620613503072, 1.2016609518017818, 0.30149574603650714, 0.30198624094386234, 0.00353064522897262, 0.22509018783099888, 0.30902691442137253, 2.0371594926807814, 3.0531172337647465]] 

[[-0.0026363191535144857, -0.004213760677085692, -0.004911502862471077, 0.24512041636642426, 0.7978181716281448, 0.10010983425111501, 0.09947561307562656, -0.002254831888500811, 0.243706764998359, 0.29827471927266486, 0

We can compute the upper and lower errors on each parameter at a given sigma limit.

Here, "ue3" signifies the upper error at 3 sigma. 

In [15]:
ue3_vectors = [
    samps.error_vector_at_upper_sigma(sigma=3.0) for samps in agg.values("samples")
]

ue3_instances = [
    samps.error_instance_at_upper_sigma(sigma=3.0) for samps in agg.values("samples")
]

le3_vectors = [
    samps.error_vector_at_lower_sigma(sigma=3.0) for samps in agg.values("samples")
]
le3_instances = [
    samps.error_instance_at_lower_sigma(sigma=3.0) for samps in agg.values("samples")
]

print("Errors Lists: \n")
print(ue3_vectors, "\n")
print(le3_vectors, "\n")
print("Errors Instances: \n")
print(ue3_instances, "\n")
print(le3_instances, "\n")

Errors Lists: 

[[0.002415688148610102, 0.0031469275555964927, 0.0041725968476822905, 0.006141525893136379, 0.0019529850148533034, 0.001923534473777036, 0.002606432948894885, 0.0048310533704796176, 0.004664678537653677, 0.012205168485982132, 0.019640539741996976, 0.04454325847905727], [0.002482055257370931, 0.002252193875361974, 0.002156085729496604, 0.0023447163284158285, 0.0017235539527848953, 0.0013688742930157427, 0.001964900337919767, 0.0045781801880421, 0.0037298104024684753, 0.010233746738688043, 0.02337720828603085, 0.02230145059150823], [0.002108360926577533, 0.002825842246738595, 0.0018938630800255374, 0.0024596613034834455, 0.0013721551237741991, 0.0012294455788831038, 0.0020304394121097302, 0.0031977691588367173, 0.0022799305213721532, 0.009346914439542053, 0.045995460637511476, 0.04260766184980369]] 

[[0.0031951965114689904, 0.0037255562239741796, 0.004136045269430144, 0.005594682982689364, 0.0015794355971422425, 0.0020122583158820456, 0.0021242039431436377, 0.00548150544

The maximum log likelihood of each model fit and its Bayesian log evidence (estimated via the nested sampling 
algorithm) are also available.

Given each fit is to a different image, these are not very useful. However, in a later tutorial we'll look at using 
the aggregator for images that we fit with many different models and many different pipelines, in which case comparing 
the evidences allows us to perform Bayesian model comparison!

In [16]:
print("Maximum Log Likelihoods and Log Evidences: \n")
print([max(samps.log_likelihoods) for samps in agg.values("samples")])
print([samps.log_evidence for samps in agg.values("samples")])

Maximum Log Likelihoods and Log Evidences: 

[5395.424574181324, 4513.498187610491, 3773.8048724201753]
[5344.9106001080945, 4459.642921416484, 3718.381155157592]


We can also print the "model_results" of all phases, which is string that summarizes every fit`s lens model providing 
quick inspection of all results.

In [17]:
results = agg.model_results
print("Model Results Summary: \n")
print(results, "\n")

Model Results Summary: 



Bayesian Evidence                                                                         5344.91060011
Maximum Likelihood                                                                        5395.42457418

Maximum Log Likelihood Model:

galaxies
    lens
        mass
            centre
                centre_0                                                                  0.001
                centre_1                                                                  0.000
            elliptical_comps
                elliptical_comps_0                                                        0.000
                elliptical_comps_1                                                        0.251
            einstein_radius                                                               0.800
    source
        bulge
            centre
                centre_0                                                                  0.102
                centre_1          

The Probability Density Functions (PDF's) of the results can be plotted using the library:

 corner.py: https://corner.readthedocs.io/en/latest/

(In built visualization for PDF's and non-linear searches is a future feature of PyAutoFit, but for now you`ll have to 
use the libraries yourself!).

(uncomment the code below to make a corner.py plot.)

In [18]:
# import corner
#
# for samples in agg.values("samples"):
#
#     corner.corner(
#         xs=samples.parameters,
#         weights=samples.weights,
#         labels=samples.model.parameter_labels,
#     )

Finished.